In [30]:
# Preamble

# Hide simple warnings
import warnings; warnings.simplefilter('ignore')

# Check environment
import sys

_python = sys.executable
_pip = "%s -m pip -q" % _python

running_at_home = "anaconda3" in _python
running_in_env = "final" in _python

# Check executable
if running_at_home and not running_in_env:
  # I have a sweet custom config at home that I like.
  raise Exception(
      "No, stop. I want to run in my custom environment, please."
  )
else:
  # We're probably in a Google Colab.
  pass

# Show executable (for my own sanity)
_python

'c:\\Users\\buzzysin\\anaconda3\\envs\\final\\python.exe'

In [31]:
# Other setup
!{_pip} install -q --upgrade pip autopep8

# 1. Environment

In [32]:
## 1.1 Fetch YoloV8

ultralytics_deps_url = "https://raw.githubusercontent.com/ultralytics/ultralytics/main/requirements.txt"
# Install dependencies
!{_pip} install -r {ultralytics_deps_url}

# Install ultralytics (yolov8)
!{_pip} install ultralytics

In [34]:
## Setup and install loggers, etc.
!{_pip} install comet_ml
!{_pip} install wandb
!{_pip} install clearml

In [59]:
## Logging and saving
from pathlib import Path

from comet_ml import Experiment
from clearml import Task
from wandb import init as wandb_init

class Conf:
  def __init__(self, path: str):
    self.path = path

  def get_conf(self, key: str, default: "str|None"= None) -> str:
    try:
      with open(self.path, "r") as f:
        lines = f.readlines()
        line = [line for line in lines if key in line][0]
        return line.split("=")[1].strip()
    except:
      return default if default is not None else ""

conf_comet = Conf(".comet.config")

# Setup comet_ml
log_comet = Experiment(
  api_key=conf_comet.get_conf("api_key"),
  project_name=conf_comet.get_conf("project_name"),
)
# log_clearml = Task.init(project_name="finalyearproject", task_name="finalyearproject")
# log_wandb = wandb_init(project="finalyearproject")


# 1.4 Download dataset

# Get the (not so) secret URL
with open("roboflow.txt", "r") as f:
  roboflow_url = f.readline().strip()

_root_path = "../../"
_datasets_path = _root_path + "datasets"
_data_path = _root_path + "data"

# Setup the directory structure
root_path = Path(_root_path)
datasets_path = root_path / "datasets"
data_path = root_path / "data"

data_path.mkdir(parents=True, exist_ok=True)
datasets_path.mkdir(parents=True, exist_ok=True)

dataset_name = "circuits"

dataset_path = datasets_path / dataset_name
dataset_path.mkdir(parents=True, exist_ok=True)

def rmrf(path: "Path | str"):
  if isinstance(path, str):
    rmrf(Path(path))
    return

  if not path.exists():
    return
  
  if path.is_file():
    path.unlink()
    return

  if path.is_dir():
    list(map(rmrf, path.iterdir()))
    path.rmdir()

COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using Experiment.log_metrics() and Experiment.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/buzzysin/finalyearproject/980018a4a4c64c84bbeeebfe0582effd



: 

In [52]:
# Images are stored in a zip file on Roboflow
!curl -sSL {roboflow_url} > {dataset_name}.zip && echo "Downloaded {dataset_name}.zip"
!unzip -q -o {dataset_name}.zip -d {dataset_path} && echo "Unzipped {dataset_name}.zip"


"Downloaded circuits.zip"
"Unzipped circuits.zip"


In [53]:
# Delete the zip file
rmrf(f"{dataset_name}.zip")

# Move the data to the correct location
data_path_bad = dataset_path / "data.yaml"
data_path_good = data_path / "data.yaml"

if data_path_bad.exists():
  if data_path_good.exists():
    data_path_good.unlink()

  data_path_bad.rename(data_path_good)
  data_path_bad.unlink(missing_ok=True)


# 2. YOLO (eugh...)

In [41]:
## Check CUDA Support

# Import ultralytics
import ultralytics

# Check to see if CUDA support is available
ultralytics.checks()

Ultralytics YOLOv8.0.51  Python-3.9.16 torch-1.13.1 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)
Setup complete  (16 CPUs, 13.9 GB RAM, 306.9/474.7 GB disk)


In [57]:
# 2.1 - Create a YOLOv8 model
from typing import Any


class my_config:
  def __init__(self, **kwargs):
    self.__dict__.update(kwargs)

  def dict(self):
    return self.__dict__.copy()
  
  def merge(self, other: "my_config|None"= None, **kwargs):
    if other is not None:
      kwargs.update(other.dict())
    
    merged = self.__dict__.copy()
    merged.update(kwargs)

    return my_config(**merged)

  def __getitem__(self, key) -> "Any":
    return self.__dict__[key]

  def __setitem__(self, key, value):
    self.__dict__[key] = value

  def __repr__(self):
    return str(self.__dict__)
  
  def __iter__(self):
    return iter(self.__dict__)
  
  def __len__(self):
    return len(self.__dict__)
  
  def __contains__(self, key):
    return key in self.__dict__
  
  def keys(self):
    return self.__dict__.keys()
  



Ultralytics YOLOv8.0.51  Python-3.9.16 torch-1.13.1 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)
yolo\engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=../../data/data.yaml, epochs=1, patience=50, batch=2, imgsz=256, save=True, save_period=5, cache=False, device=0, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, min_memory=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False

2023-03-09 19:49:46,015 - clearml.model - INFO - Selected model id: 7f7ee960722b401ba7d1022448100aa2


Model summary (fused): 168 layers, 3006428 parameters, 0 gradients, 8.1 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:04<00:00,  1.19it/s]
                   all         20        248          0          0          0          0
Speed: 1.6ms preprocess, 8.0ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs\detect\train6
Ultralytics YOLOv8.0.51  Python-3.9.16 torch-1.13.1 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)
Model summary (fused): 168 layers, 3006428 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning C:\Users\buzzysin\code-windows\y4\final-project\datasets\circuits\valid\labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:03<00:00,  2.63it/s]
                   all         20        248          0          0          0

In [ ]:
from ultralytics import YOLO

# or yolov8s.pt, yolov8m.pt, yolov8l.pt, yolov8x.pt
model = YOLO()  

config_base = my_config(
    data=data_path.joinpath("data.yaml").as_posix(),
    epochs=5,
    batch=16,
    imgsz=512,
    save=True,
    save_period=5,
    device="0")

configs = my_config(
    test=my_config(**config_base.dict()),
    production=my_config(**config_base.merge(epochs=100, batch=32, imgsz=1024).dict()),
    debug=my_config(**config_base.merge(epochs=1, batch=2, imgsz=256).dict())
)

# 2.2 - Train the model
results = model.train(**configs.debug.dict()) # type: ignore
validated = model.val()

In [58]:
# Test the model
# validation = model.val()
log_comet.end()

COMET ERROR: Couldn't log ipython notebook
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/buzzysin/finalyearproject/fd1f12750866472292b6e934bd78651d
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     metrics/mAP50(B)     : 0.0
COMET INFO:     metrics/mAP50-95(B)  : 0.0
COMET INFO:     metrics/precision(B) : 0.0
COMET INFO:     metrics/recall(B)    : 0.0
COMET INFO:     model/GFLOPs         : 8.197
COMET INFO:     model/parameters     : 3011628
COMET INFO:     model/speed(ms) [2]  : (8.0, 12.751)
COMET INFO:     train/box_loss       : 2.34714
COMET INFO:     train/cls_loss       : 4.37232
COMET INFO:     train/dfl_loss       : 1.29552
COMET INFO:     val/box_loss      

In [14]:
# Predict on a video

import cv2 as cv

model = YOLO("runs/detect/train2/weights/best.pt")

live_stream = cv.VideoCapture(0)
live_stream_size = (
    int(live_stream.get(cv.CAP_PROP_FRAME_WIDTH)),
    int(live_stream.get(cv.CAP_PROP_FRAME_HEIGHT)))

print(live_stream_size)

cv.namedWindow("frame", cv.WINDOW_NORMAL)
cv.setWindowProperty("frame", cv.WND_PROP_TOPMOST, 1)
cv.setWindowProperty("frame", cv.WND_PROP_AUTOSIZE, cv.WINDOW_AUTOSIZE)
cv.setWindowProperty("frame", cv.WND_PROP_ASPECT_RATIO, cv.WINDOW_KEEPRATIO)
cv.setWindowProperty("frame", cv.WND_PROP_OPENGL, cv.WINDOW_OPENGL)
cv.moveWindow("frame", 0, 0)

# stream_save = cv.VideoWriter("./live_stream_predict.avi", -1, 20.0, live_stream_size)
# fourcc = cv.VideoWriter_fourcc(*'mp4v')
# stream_save = cv.VideoWriter("./live_stream_predict.mp4", fourcc, 20.0, live_stream_size)

if not live_stream.isOpened():
  print("Cannot open camera")
  raise Exception("Cannot open camera")

# if not stream_save.isOpened():
#   print("Cannot open video writer")
#   raise Exception("Cannot open video writer")

try:
  # Get the video
  # live_stream = cv.VideoCapture("http://192.168.50.25:8080/video")

  frame_no = 0
  while True:
    ret, frame = live_stream.read()

    if not ret:
      break

    predictions = model.predict(frame, imgsz=512, conf=0.75, classes=[0,1,3])

    write_frame = None

    if len(predictions) == 0:
      write_frame = frame
    else:
      write_frame = predictions[0].plot()

    # stream_save.write(write_frame)
    cv.imshow("frame", write_frame)
    cv.waitKey(1)
    

  # Predict on the video
  # predictions = model.predict(source=live_stream, show=True)
finally:
  live_stream.release()
  # stream_save.release()
  cv.destroyAllWindows()


(640, 480)



0: 384x512 (no detections), 18.0ms
Speed: 13.0ms preprocess, 18.0ms inference, 7.0ms postprocess per image at shape (1, 3, 512, 512)

0: 384x512 (no detections), 32.5ms
Speed: 1.0ms preprocess, 32.5ms inference, 2.0ms postprocess per image at shape (1, 3, 512, 512)

0: 384x512 (no detections), 15.0ms
Speed: 1.0ms preprocess, 15.0ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 512)

0: 384x512 (no detections), 17.5ms
Speed: 1.0ms preprocess, 17.5ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 512)

0: 384x512 (no detections), 16.0ms
Speed: 1.0ms preprocess, 16.0ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 512)

0: 384x512 (no detections), 14.0ms
Speed: 1.0ms preprocess, 14.0ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 512)

0: 384x512 (no detections), 14.0ms
Speed: 1.0ms preprocess, 14.0ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 512)

0: 384x512 (no detections), 21.0ms
Speed: 1.0ms preprocess, 21.0ms 

KeyboardInterrupt: 